# Classification of Galaxies According to their Structure(Morphology) using the Galaxy Zoo Project Dataset

## Background

Galaxy classification is fandamental when investigating properties of galaxies in statistical analysis in order to understand their evolution prosess. The galaxies are classified following the Hubble classifications as either spirals, ellipticals or irregulars if they do not fit in any of the previous classifications. Traditionally, galaxies are classified by visualy inspecting galaxy images. With the increase in the number of observed samples, visual classification is not only time consuming or labour intensive, but nearly imposible to classify large galaxy samples. Their is a need to automate the classifications that can replace the visual classifications with high accuracy . 

The Galaxy Zoo Project 2 classfied around 200,000 galaxy images obtained from the Sloan Digital Sky Survey (SDSS), DR7 and other similar surveys. The images are classified into respective categories (37) following a set of 11 questions with the help of the public (citizen scientist).

The project used the Galaxy Zoo dataset from kaggle consisting of 61,578 galaxy images in the training dataset and 79,973 images for the test dataset.

### Overview

The main aim of this project is to build a model that can automate the galaxy classification in to two main categories (spirals and ellipticals) with very high accuracy.

This notebook contain the following sections,
- Data Exploration
- Selection Method
- Data Preprocessing
- Methodology
- Model Training 
- Model Visualization
- Conclusions and Recommendations



## Exploring and Understanding the Dataset

While the traing dataset contain 61,578 images, the training_solution_rev1.csv file has the information regarding the probablities of a galaxies belonging to the respective classes. From the 37 classes, the main categories of interest are classes 1.1 and 1.2. Based on the first classification question, class 1.1 give the probability of a galaxy being smooth (elliptical) and class 1.2 gives the probability of it having features or disk(spirals). 

For a more clean sample, we used the images classified with a more than 0.8 probability as belonging to the two classes and labeling then appropriatly. Out of all the traing dataset images, only 24,273 satisfied our requement (>= 0.8 probability in either class 1.1 or 1.2). We separated the data that satisfied our requarements into a new dataframe. When inspecting the new dataframe, no duplicates or missing values were found.


In [ ]:
#importing required libraries for data exploration
import pandas as pd
import numpy as np

In [ ]:
#reading the training csv file and displaying the first 5 row 
#to have a better understanding of the classes
data = pd.read_csv("../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip",
                   compression="zip")

#data = pd.read_csv('training_solutions_rev1.csv')
data.head(5)

In [ ]:
#looking for the number of rows and colunm in the dataframe
data.shape

We define a fuction that will be used to add the image type extection to the GalaxyID. Since our images are 'jpg', we will use this extection in the defined fuction. The function is then used to add the extection on the dataframe colunm GalaxyID.

In [ ]:
#definig a function to add image extections to the GalaxyID
#adding the image extections to the GalaxyID

def append_ext(filename):
    """ Appends `.jpg` file extension to a filename """
    return f"{filename}.jpg"
    #return filename + '.jpg'

#applying the function to the GalaxyID colunm
data["GalaxyID"] = data["GalaxyID"].apply(append_ext)

#printing the first five row of the dataframe to view the changes
data.head(5)

In [ ]:
#creating a new dataframe with galaxies having a probability of 0.8 in either class1.1 
#or class1.2
new_data = data[((data["Class1.1"] >= .8) | (data["Class1.2"] >= .8))].reset_index(drop=True)
print(new_data.shape)
new_data.head(5)

In [ ]:
#looking for duplicate data
new_data.drop_duplicates(inplace=True)
new_data.shape

#the are no duplicate images in the dataset


In [ ]:



#inspecting missing data
new_data.isnull().sum().sum()

In [ ]:
#adding a new colunm with labels for the galaxies with probability greater than 0.8.
#we label those in class1.1 as ellipticals and those in class1.2 as spirals

#define conditions
conditions = [new_data["Class1.1"] >= 0.8, 
              new_data["Class1.2"] >= 0.8]

#define choices
choices = ["elliptical", "spiral"]

#create new column in DataFrame that displays results of comparisons
new_data["class"] = np.select(conditions, choices, default="Tie")

new_data

In [ ]:
#Exploring the distributing of the new dataframe
print(new_data[new_data["class"] == "elliptical"].shape)

print(new_data[new_data["class"] == "spiral"].shape)

## Selection Method
From the new dataframe with elliptical and spiral labels (24,273), we have 8,132 elliptical galaxy images and 16,141 spirals. Since our main interest is on the model accuracy, it is important to balance the number of sample used from both classes to prevent the model from learning to classify only one class. To balance the training dataset, we selected 8000 random samples from each of the two categories. However we first copied a small sample of our data, 500 images from each of the two class, to a new folder for visual inspection. This sample was later manually cropped and used for the Minimum Valuable Product (MVP).

We at some point considered the inclusion of a few antifacts to improve model traing in identifing what is not a spiral or elliptical galaxy. The antifacts category corresponds to Class 1.3. However, this was not included in the final product.

In [ ]:
#sort the main data by class 1.3 (antifacts)
data.sort_values("Class1.3", 
                 ascending=False,
                 inplace=True)
data.head(10)

Antifacts classifed with a percentage fraction of 0.8 are only 2. Those with a percentage fraction of 0.6 are 24 and 44 have a fraction of 0.5. It would have been important to visually inspect this subset of objects to ensure they a trully antifacts and will not mislead the model.

In [ ]:
#printing out samples of antifacts based on specified selections

#data[data["Class1.3"] >= .8 ] #only 2
#data[data["Class1.3"] >= .6 ].shape # 24
data[data["Class1.3"] >= .5 ] #44

In [ ]:
#dataframe for the antifacts
df_antifacts = data[data["Class1.3"] >= .5 ]
df_antifacts.shape

## Data Preprocessing

Since our model was to mainly classify the spirals and the elliptical galaxy images, we procceded with the selection and preprocessing stage in preparation for model training. For the MVP, only 1000 galaxy samples were selected (500 for both the spiral and elliptical galaxaies). We copied the images into a folder in the local machine for visual verification and cropping with the code below.

For the final product a total of 16,000 images were randomly selected as a subset of the filtered data.

In [ ]:
# randomly selecting 500 samples in each category for the mvp and 
# saving it in to a new dataframe
df_spiral_new = new_data[new_data["class"] == "spiral"].sample(n=500)
print("This is a sample of spiral galaxies")
print(df_spiral_new.head(5))
print(df_spiral_new.shape)

df_elliptical_new =new_data[new_data["class"] == "elliptical"].sample(n=500)
print("And this is a sample of elliptical galaxies")
print(df_elliptical_new.head(5))
print(df_elliptical_new.shape)

The cell below was previously used to copy images based on a dataframe into diffent folders for the mvp model training. We created new directories for the defferent categories, defined the source and destination directories and finally looped over your desired files from the dataframe and copying the content of the source directiry to the desired destination.

In [ ]:
# # importing shutil module
# import os
# from shutil import copyfile

# #creating new directories for the defferent categories
# os. mkdir("/home/marcelina/Desktop/DSI/project/data/spirals") 
# os. mkdir("/home/marcelina/Desktop/DSI/project/data/ellipticals") 

# # define your source and destination directories
# #spriral
# source_directory1 = "/home/marcelina/Desktop/DSI/project/images_training_rev1"
# destination_directory1 = "/home/marcelina/Desktop/DSI/project/data/spirals"
# #elliptical
# # source_directory2 = "/home/marcelina/Desktop/DSI/project/images_training_rev1"
# destination_directory2 = "/home/marcelina/Desktop/DSI/project/data/ellipticals"


# # loop over your desired files from the dataframe to copy them
# # Copy the content of source to destination
# for fileName in df_spiral_new["GalaxyID"]: 
#     source_file1 = source_directory1 + "/" + fileName 
#     destination_file1 = destination_directory1 + "/" + fileName
#     #copyfile(source_file1, destination_file1)


# for fileName in df_elliptical_new["GalaxyID"]: 
#     source_file2 = source_directory2 + "/" + fileName 
#     destination_file2 = destination_directory2 + "/" + fileName
#     #copyfile(source_file2, destination_file2)


### Preprocessing for the final product

In [ ]:
# selecting equal number of spiral
# and elliptical galaxies
# randomly selecting 8,000 samples in each categiry
df_spiral = new_data[new_data["class"] == "spiral"].sample(n=8000,random_state = 42)
print("This is a sample of spiral galaxies")
print(df_spiral.head(5))
print(df_spiral.shape)

df_elliptical =new_data[new_data["class"] == "elliptical"].sample(n=8000,random_state = 42)
print("And this is a sample of elliptical galaxies")
print(df_elliptical.head(5))
print(df_elliptical.shape)

In [ ]:
#merging the two dataframes together to make the traing dataframe
#sellecting only the galaxyid and classes columns
frames = [df_spiral, df_elliptical]
df_galaxies = pd.concat(frames)
df_galaxies = df_galaxies[["GalaxyID", "class"]]
df_galaxies.head(5)

In [ ]:
# printing the shape of the final dataframe
df_galaxies.shape

In [ ]:
#unziping the test image folder
import zipfile
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# visualing one of the images and the image size
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

#images
img= image.load_img("./images_training_rev1/104934.jpg")
img2 = image.load_img("./images_training_rev1/582003.jpg")

#Subplots
plt.figure(figsize=(12,4.5))
plt.subplot(141)
plt.imshow(img)
plt.title("Spiral Galaxy", y=1.02, fontsize=12)

plt.subplot(142)
plt.imshow(img2)
plt.title("Elliptical Galaxy", y=1.02, fontsize=12)

;

In [ ]:
import cv2

print("The images are of size: ")
cv2.imread("./images_training_rev1/104934.jpg").shape

#the images have the shape 424,424,3

## Methodology
For the mvp, the 1000 cropped images were used to train the model while trying both gray scale and the RGB scale. The models trained with the RGB scale performed better which showed the importance of colour in galaxy classifcations. Unfortunately, it was harder to get a good performance with such a small sample. We later included the earlier selected sample of 16,000 galaxies in RGB colour as the training sample and tried optimizing on the number of layers, filters and other parameters to improve the model performance.

To prepare the data for model training we used the ImageDataGenerator and the training dataframe for the image id and the class columns to lable the images. We resized the image size to 212 x 212, used the RGB colour and a batch_size of 32. The training (80%) and the validation (20%) datasets were separated by the 80-20 rule. We randomly shuffled the images so that the model will not learn to classfy one category more. The elliptical galaxies have the zero lable (0) while the spiral galaxies have the one (1).

The model that gave us an optimum performance was a cnn model with seven layers which include an input layer, three fully connected layers and the final three dense layers with relu activation functions for all layers as shown in the model summary. We used the categorical_crossentropy loss function, Adam optimizer with a learning rate of 0.001 and measured the accuracy. Checkpoints and early stoppings where used to prevent the model from overfitting and only saving the best model.

## Model Training

In [ ]:
#importing libraries for data visualization and model training

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import preprocess_input

import numpy as np
import os
#from google.colab.patches import cv2_imshow # cv2.imshow does not work on 
#Google Colab notebooks, which is why we are using cv2_imshow instead

In [ ]:
# Using the ImageDataGenerator to prepare the data for training
datagen = ImageDataGenerator(rescale=1./255.,
                           validation_split=0.2,

)


train_generator = datagen.flow_from_dataframe(dataframe = df_galaxies,
                                          target_size = (212,212),
                                          directory = "./images_training_rev1",
                                          batch_size = 32,
                                          x_col="GalaxyID",
                                          y_col="class",
                                          color_mode="rgb",
                                          class_mode="categorical",
                                          subset="training",
                                          shuffle=True,
                                          seed=42
                                         )
valid_generator = datagen.flow_from_dataframe(dataframe = df_galaxies,
                                            target_size=(212, 212),
                                            directory = "./images_training_rev1",
                                            batch_size=32,
                                            x_col="GalaxyID",
                                            y_col="class",
                                            color_mode="rgb",
                                            class_mode="categorical",
                                            subset="validation",
                                            shuffle=True,
                                            seed=42
                                           )                                        

In [ ]:

#printing the generated class and the index class
train_generator.class_indices

In [ ]:
#the cnn model architecture
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(6,6),activation = 'relu',
                                                           input_shape=(212,212,3)),
                                  #tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  #tf.keras.layers.Dropout(0.1),
                                    
                                  #
                                  tf.keras.layers.Conv2D(32,(5,5),activation = 'relu'),
                                  #tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Dropout(0.1),
                                    
                                  #
                                  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                  #tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  #tf.keras.layers.Dropout(0.1),
                                    
                                  #
                                  tf.keras.layers.Conv2D(128,(3,3),activation = 'relu'),
                                  #tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Dropout(0.1),
                                    
                                  #
                                  tf.keras.layers.Flatten(),
                                  
                                  #
                                  tf.keras.layers.Dense(512, activation='relu'),
                                  #tf.keras.layers.Dropout(0.1),
                                    
                                  tf.keras.layers.Dense(256, activation='relu'),
                                  #tf.keras.layers.Dropout(0.1),
                                    
                                   #
                                  tf.keras.layers.Dense(2, activation='softmax')
                                  ])

In [ ]:
#printing and saving the model summary and architecture
summary = model.summary()
tf.keras.utils.plot_model(model, to_file="model_plot3.png", 
                          show_shapes=True, show_layer_names=True)

In [ ]:
#creating a checkpoint to save the best model
checkpoint = ModelCheckpoint("best_model3.h5", 
                             monitor = "val_loss",
                             verbose = 0, 
                             save_best_only = True, 
                             mode = "auto")
#Specifying an earlystopping to avoid overfitting
stopping = EarlyStopping(monitor = "val_loss",
                         patience = 10,
                         verbose = 0,
                         mode = "auto")

In [ ]:
#compliling the model 
model.compile(
loss="categorical_crossentropy",
optimizer=Adam(learning_rate=0.001),
metrics=["accuracy"])


In [ ]:
#model fitting
#hist = model.fit(train_generator, epochs = 30, validation_data=valid_generator, 
#                 callbacks = [checkpoint, stopping])



#To avoid retraing the model again, we will upload the downloded model 
#loadig the trained model
model = tf.keras.models.load_model("../input/mymodel/best_model3.h5")

The code below was used to plot the training history that has been displayed as an image in the cell after.

In [ ]:
# #creating subplots for traing plots
# #fig, axes = plt.subplots(1, 2, figsize=(12,6))

# # summarize history for accuracy
# #axes[0].plot(hist.history['accuracy'])
# #axes[0].plot(hist.history['val_accuracy'])
# axes[0].set_ylabel('accuracy')
# axes[0].set_xlabel('epoch')
# axes[0].legend(['train', 'test'], loc='upper left')


# # summarize history for loss
# #axes[1].plot(hist.history['loss'])
# #axes[1].plot(hist.history['val_loss'])
# axes[1].set_ylabel('loss')
# axes[1].set_xlabel('epoch')
# axes[1].legend(['train', 'test'], loc='upper left')

# #fig.title('Model Accuracy')
# #plt.savefig('model_Accuracy_plot.png')
# #plt.show()


In [ ]:
#seens the model is now preloaded we will plot
img3= image.load_img("../input/the-model-training-graph/Screenshot from 2022-05-09 12-34-35.png")
plt.figure(figsize=(20,10))
plt.imshow(img3)

In [ ]:
# #getting a link to dowmload the the model and the plots 
# os.chdir(r"../working")
# from IPython.display import FileLink

# FileLink(r"model_Accuracy_plot.png")

# FileLink(r"model_plot3.png")

# FileLink(r"best_model3.h5")
         


### Testing the model with a new dataset not used in training

In [ ]:
#Trying to obtain the rows that were not in the traing set 
#for final testing of our trained model

# we start by appening the traing datast in the whole dataframe 
#for spirals and elliptical galaxies 
df_spiral_all = new_data.append(df_spiral)
df_all = df_spiral_all.append(df_elliptical)

#drop rows which contain duplicates, Drop all duplicates.
df_test_all = df_all.drop_duplicates(keep=False)
df_test_all.shape


In [ ]:
# printing the shape of the dataframe for the specific categories
print(df_test_all[df_test_all["class"] == "spiral"].shape)
print(df_test_all[df_test_all["class"] == "elliptical"].shape)

In [ ]:
#Preparing an equal sample of text data for the model
df_spiral_test = df_test_all[df_test_all["class"] == "spiral"].sample(n=130,
                                                                      random_state = 42)
#
df_elliptical_test = df_test_all[df_test_all["class"] == "elliptical"].sample(
    n=130, random_state = 42)

#merging the two dataframes together to make the testing dataframe
#sellecting only the galaxyid and classes colunms
frame = [df_spiral_test, df_elliptical_test]
df_test_new = pd.concat(frame)
df_test_new = df_test_new[["GalaxyID", "class"]]
df_test_new.shape


In [ ]:
#labling the test images with ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.,
                           )

test_generator = datagen.flow_from_dataframe(dataframe = df_test_new,
                                          target_size = (212,212),
                                          directory = "./images_training_rev1",
                                          batch_size = 32,
                                          x_col="GalaxyID",
                                          y_col="class",
                                          color_mode='rgb',
                                          class_mode='categorical',
                                          shuffle=True,
                                          seed=42)

In [ ]:
#Printing the loss value and the accuracy for the evaluation with test data
score = model.evaluate(test_generator, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


## Model Visualization

In [ ]:
# printing the sumary 
model.summary()

#### Visualizing a disc like spiral galaxies

In [ ]:
# uploading an image for prediction
img_upload = "../input/image-next/images.jpeg"

img = image.load_img(img_upload, target_size=(212, 212))

plt.imshow(cv2.imread(img_upload));# Visualize image

In [ ]:
#the first class is elliptical while the second class is spiral
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#making a prediction
preds = model.predict(x)
print(preds)

In [ ]:
# Defining a fuction that takes an input and outputs a 
#Gradient Class Activation Map for the final spacified layer
with tf.GradientTape() as tape:
  last_conv_layer = model.get_layer("conv2d_31")
  iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
  model_out, last_conv_layer = iterate(x)
  class_out = model_out[:, np.argmax(model_out[0])]
  grads = tape.gradient(class_out, last_conv_layer)
  pooled_grads = K.mean(grads, axis=(0, 1, 2))
  
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)

In [ ]:
#visualizing the heat map

heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
heatmap = heatmap.reshape((21, 21))
plt.matshow(heatmap)
plt.savefig("Astromomy.png")
plt.show()

In [ ]:
#getting a link to dowmload the the heatmap image
os.chdir(r"../working")
from IPython.display import FileLink
FileLink(r"Astromomy.png")

#### Visualizing an elliptical galaxy

In [ ]:
# uploading an image for prediction
img_upload = "../input/elliptical/Elliptical_galaxy.jpg"

img = image.load_img(img_upload, target_size=(212, 212))

plt.imshow(cv2.imread(img_upload));# Visualize image

In [ ]:
#the first class is elliptical while the second class is spiral
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#making a prediction
preds = model.predict(x)
print(preds)

In [ ]:
# Defining a fuction that takes an input and outputs a 
#Gradient Class Activation Map for the final spacified layer
with tf.GradientTape() as tape:
  last_conv_layer = model.get_layer("conv2d_31")
  iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
  model_out, last_conv_layer = iterate(x)
  class_out = model_out[:, np.argmax(model_out[0])]
  grads = tape.gradient(class_out, last_conv_layer)
  pooled_grads = K.mean(grads, axis=(0, 1, 2))
  
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)

In [ ]:
#visualizing the heat map

heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
heatmap = heatmap.reshape((21, 21))
plt.matshow(heatmap)
plt.savefig("Astromomy2.png")
plt.show()

In [ ]:
#getting a link to dowmload the the heatmap image
os.chdir(r"../working")
from IPython.display import FileLink
FileLink(r"Astromomy2.png")

## Conclusion and Recommendations
### Conclusion
The model architecture seem to perform considerably well for our sample with an optimized  validation and test accuracy within the range (96 - 98)%. More augmentation is needed for the model to generalise well. However,from random image predictions and plotting the heat maps form the model, it seemed that most of the galaxies well classified as spiral galaxies were the disk like spiral galaxy type (See the first image and the corresponding image heat map in the Model Visualization). Surprisingly, galaxies with very well defined and observed arms are strongly mis-classified as elliptical. But going back to the images, it was evident that our selected sample of spriral galaxies was heavily bias towards the disk like structures. The small number of spiral galaxies were maybe consided outliers by our model.

### Recommendations
- Based on the type of galaxies classifed as spiral glaxies, it maybe important for the sample of spiral galaxies to be balanced between the two or more classifications added for an improvement in the classifications.
- The traing data could be increased by including the galaxies with a lower percentage fraction in our selection criteria.
- Linear based models can be used to compare our model perfomance with.